# Aproksymacja Średniokwadratowa Wielomianami Algebraicznymi

Implementacja aproksymacji średniokwadratowej dla funkcji $f(x) = x^2 - m \cos\left(\frac{\pi x}{k}\right)$ na zadanym przedziale, w oparciu o dyskretne punkty próbkowania.

In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from collections.abc import Callable
import ipympl
import math
%matplotlib ipympl

## Konfiguracja i Ustawienia

In [ ]:

m = 5.0 
k = 0.5 
interval = (-5.0, 5.0) 
func = lambda x: x**2 - m * np.cos((np.pi * x) / k)

N = 50
MAX_POLYNOMIAL_DEGREE = 15 
NUMBER_OF_PROBES = 1000

PATH_TO_SAVE_MAIN_APPROX = os.path.join('.', 'img_approximation')
PATH_TO_SAVE_UNIFORM_APPROX = os.path.join(PATH_TO_SAVE_MAIN_APPROX, 'uniform')
PATH_TO_SAVE_CHEBYSHEV_APPROX = os.path.join(PATH_TO_SAVE_MAIN_APPROX, 'chebyshev')
PATH_TO_SAVE_DATA_APPROX = os.path.join('.', 'data_approximation')


## Funkcje Pomocnicze

In [ ]:
def generate_function_uniform_nodes(n: int, interval: tuple[np.float64, np.float64], func: Callable[[np.float64], np.float64]) -> np.ndarray:
  a,b= interval[0], interval[1]
  xs = np.linspace(a,b,n)
  tups = []
  for x in xs:
    tups.append((x,func(x)))
  return np.array(tups, dtype=np.float64)

def get_max_error(func: Callable[[np.float64], np.float64], approximation_func: Callable[[np.float64], np.float64], interval: tuple[np.float64, np.float64]) -> np.float64:
  xs = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES)
  return np.max([np.abs(func(x) - approximation_func(x)) for x in xs])

def get_mean_squared_error(func: Callable[[np.float64], np.float64], approximation_func: Callable[[np.float64], np.float64], interval: tuple[np.float64, np.float64]) -> np.float64:
  xs = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES)
  return np.sqrt(np.sum([((func(x) - approximation_func(x))**2) for x in xs])) / NUMBER_OF_PROBES



# Wizualizacja

In [ ]:
def plot_function(
  x_values: np.ndarray[np.float64],
  func_to_plot: Callable[[np.float64], np.float64],
  nodes: np.ndarray | None = None,
  x_lim: tuple[float, float] | None = None,
  y_lim: tuple[float, float] | None = None,
  x_scale: str = 'linear',
  y_scale: str = 'linear',
  function_name: str = 'Funkcja bazowa',
  nodes_name: str = 'Punkty aproksymacji',
  title: str = 'Wizualizacja funkcji i aproksymacji',
  linestyle: str = '-', 
  color: str | None = None,
  alpha: float = 1.0
):
  y_values = np.array([func_to_plot(x) for x in x_values])

  if(color):
    plt.plot(x_values, y_values, label=function_name,
            linestyle=linestyle, color=color, alpha=alpha)
  else:
    plt.plot(x_values, y_values, label=function_name,
            linestyle=linestyle, alpha=alpha)
  
  if nodes is not None:
    if(color):
      plt.scatter(nodes[:, 0], nodes[:, 1], color=color,
                  label=nodes_name, s=15, zorder=5)
    else:
      plt.scatter(nodes[:, 0], nodes[:, 1],
                  label=nodes_name, s=15, zorder=5)

  plt.xlabel('x')
  plt.ylabel('y')

  if x_lim:
    plt.xlim(x_lim)
  if y_lim:
    plt.ylim(y_lim)
  
  plt.xscale(x_scale)
  plt.yscale(y_scale)

  plt.legend()
  plt.title(title)
  plt.grid(True)


## Implementacja Aproksymacji Średniokwadratowej

In [ ]:
def get_poly_approximation_func(nodes: np.ndarray, degree: int) -> Callable[[np.float64], np.float64]:

  m = degree
  
  if(len(nodes) < m + 1):
    raise Exception("Number of nodes must be greater than degree")

  x_coords = nodes[:, 0]
  y_coords = nodes[:, 1]

  x_powers_sums = [np.sum(x_coords**k) for k in range(2 * m + 1)]

  gram_matrix = np.zeros((m + 1, m + 1), dtype=np.float64)
  for j in range(m + 1):
    for k in range(m + 1):
      gram_matrix[j, k] = x_powers_sums[j + k]

  moment_vector = np.zeros(m + 1, dtype=np.float64)
  for j in range(m + 1):
    moment_vector[j] = np.sum(y_coords * (x_coords**j))

  coefficients = np.linalg.solve(gram_matrix, moment_vector)

  def polynomial(x: np.float64) -> np.float64:
    return sum(coefficients[i] * x**i for i in range(m + 1))

  return polynomial


# Implementacja Aproksymacji Trygonometrycznej

In [ ]:
def get_trig_approximation_func(nodes: np.ndarray, degree: int) -> Callable[[np.float64], np.float64]:

  m = degree

  x_coords = nodes[:, 0]
  y_coords = nodes[:, 1]
  n = len(x_coords) 
  
  v_coords = ((x_coords + 5)/10) * 2 * np.pi

  def a_k(k):
    return 2/n * sum([(y_coords[i]) * np.cos(k * v_coords[i]) for i in range(n)])
  def b_k(k):
    return 2/n * sum([(y_coords[i]) * np.sin(k * v_coords[i]) for i in range(n)])

  
  def approx(x):
    x_1 = ((x+5)/10) * 2 * np.pi
    return a_k(0)/2 + sum([a_k(k) * np.cos(k * x_1) + b_k(k) * np.sin(k * x_1) for k in range(1,m+1)])

  return approx


## Przykładowe Użycie (Jeden Stopień Wielomianu)

In [ ]:
degree = 20

sample_nodes_uniform = generate_function_uniform_nodes(N, interval, func)

approximating_poly = get_poly_approximation_func(sample_nodes_uniform, degree)

max_err_example = get_max_error(func, approximating_poly, interval)
mse_example = get_mean_squared_error(func, approximating_poly, interval)

print(f"Approximation Degree: {degree}")
print(f"Number of Sample Points: {N}")
print(f"Max Error: {max_err_example:.6e}")
print(f"MSE: {mse_example:.6e}")

plt.figure(figsize=(10, 6)) 
fine_x = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES)
plot_function(fine_x, func, sample_nodes_uniform, y_lim=(-10, 40), function_name='Funkcja Oryginalna', nodes_name=f'{N} Punktów Próbkowania (Równoodl.)')
plot_function(fine_x, func, sample_nodes_uniform, y_lim=(-10, 40), function_name='Funkcja Oryginalna', nodes_name=f'{N} Punktów Próbkowania (Równoodl.)')
plot_function(fine_x, approximating_poly, None, y_lim=(-10, 40), function_name=f'Aproksymacja Wielomianowa (st. {degree})')
plt.title(f'Aproksymacja Średniokwadratowa (Stopień {degree}, {N} punktów)')
plt.show()
print("----------------------\n")

In [ ]:
degree = 15

sample_nodes_uniform = generate_function_uniform_nodes(N, interval, func)

approximating_trig = get_trig_approximation_func(sample_nodes_uniform, degree)

max_err_example = get_max_error(func, approximating_trig, interval)
mse_example = get_mean_squared_error(func, approximating_trig, interval)

print(f"Approximation Degree: {degree}")
print(f"Number of Sample Points: {N}")
print(f"Max Error: {max_err_example:.6e}")
print(f"MSE: {mse_example:.6e}")

plt.figure(figsize=(10, 6)) 
fine_x = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES)
plot_function(fine_x, func, sample_nodes_uniform, y_lim=(-10, 40), function_name='Funkcja Oryginalna', nodes_name=f'{N} Punktów Próbkowania (Równoodl.)')
plot_function(fine_x, approximating_trig, None, y_lim=(-10, 40), function_name=f'Aproksymacja Wielomianowa (st. {degree})')
plt.title(f'Aproksymacja Średniokwadratowa (Stopień {degree}, {N} punktów)')
plt.show()
print("----------------------\n")

## Wizualizacja
Tak po prawdzie to tutaj już psycha mi siadła i wizualizacja powstała ze szczególną pomocą modelu językowego

In [ ]:

# --- Configuration for Batch Processing ---
N_NODES = [10, 20, 50, 100] # List of numbers of nodes to test

# Define degrees specific to each N_NODES entry
M_DEGREES_LIST = [                
  [2,3,4],                 # For N=10
  [2,3,4, 7, 8, 9],             # For N=20
  [7, 8, 9, 15, 20, 24],             # For N=50
  [2, 3, 4, 7, 8, 9, 15, 20, 24, 30, 40, 49] # For N=100 (Adjusted example)
]

# Configuration for grouped comparison plots
GROUP_SIZE = 3
COLORS = ['green', 'red', 'blue', 'orange', 'brown', 'pink'] # Colors to cycle through within a group plot

# Define paths (Ensure PATH_TO_SAVE_MAIN_APPROX and PATH_TO_SAVE_DATA_APPROX are defined earlier)
PATH_TO_SAVE_UNIFORM_POLY_APPROX = os.path.join(PATH_TO_SAVE_MAIN_APPROX, 'uniform_polynomial_individual')
PATH_TO_SAVE_UNIFORM_TRIG_APPROX = os.path.join(PATH_TO_SAVE_MAIN_APPROX, 'uniform_trigonometric_individual')
PATH_TO_SAVE_COMPARISON_POLY = os.path.join(PATH_TO_SAVE_MAIN_APPROX, 'uniform_polynomial_comparison_grouped') # Updated path
PATH_TO_SAVE_COMPARISON_TRIG = os.path.join(PATH_TO_SAVE_MAIN_APPROX, 'uniform_trigonometric_comparison_grouped') # Updated path

# --- Cleanup and Directory Creation ---
# Create necessary directories
os.makedirs(PATH_TO_SAVE_UNIFORM_POLY_APPROX, exist_ok=True)
os.makedirs(PATH_TO_SAVE_UNIFORM_TRIG_APPROX, exist_ok=True)
os.makedirs(PATH_TO_SAVE_COMPARISON_POLY, exist_ok=True)
os.makedirs(PATH_TO_SAVE_COMPARISON_TRIG, exist_ok=True)
os.makedirs(PATH_TO_SAVE_DATA_APPROX, exist_ok=True)

# --- Data Collection Setup ---
data_records_approx = []

# --- Helper for Formatting Floats in CSV ---
def format_float_approx(value):
  try:
    num = float(value)
    if not np.isfinite(num):
      if np.isinf(num): return 'inf' if num > 0 else '-inf'
      elif np.isnan(num): return 'nan'
      else: return str(value)
    if abs(num) >= 1e6 or (abs(num) < 1e-4 and num != 0.0):
      return f'{num:.6e}'
    else:
      return f'{num:.6f}'
  except (ValueError, TypeError):
    return value

# --- Main Loop for Approximation, Plotting, and Data Collection ---
print("Starting batch approximation process...")
fine_x = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES)
y_lim_plot = (-10, 40) # Consistent y-limits for plots

for i, n_nodes in enumerate(N_NODES):
    print(f"Processing N = {n_nodes} nodes...")
    if i >= len(M_DEGREES_LIST):
        print(f"  WARNING: No degree list defined for N={n_nodes}. Skipping.")
        continue

    current_degrees = M_DEGREES_LIST[i]
    print(f"  Using Degrees: {current_degrees}")

    # Generate uniform nodes
    try:
      nodes_uniform = generate_function_uniform_nodes(n_nodes, interval, func)
      nodes_generated = True
    except Exception as e:
        print(f"  ERROR generating nodes for N={n_nodes}: {e}. Skipping this N.")
        nodes_generated = False
        continue

    # Dictionaries to store functions for comparison plots for this n_nodes
    poly_funcs_for_n = {}
    trig_funcs_for_n = {}

    for degree in current_degrees:
        print(f"    Processing Degree = {degree}...")

        # --- Polynomial Approximation ---
        poly_success = False
        try:
            # Add check: degree must be < n_nodes for meaningful polynomial least squares fit
            if degree >= n_nodes:
                 print(f"      WARNING: Polynomial degree {degree} >= number of nodes {n_nodes}. Skipping poly calculation.")
                 poly_max_err = np.nan
                 poly_mse = np.nan
            else:
                poly_approx_func = get_poly_approximation_func(nodes_uniform, degree)
                poly_max_err = get_max_error(func, poly_approx_func, interval)
                poly_mse = get_mean_squared_error(func, poly_approx_func, interval)
                poly_success = True
                poly_funcs_for_n[degree] = poly_approx_func
        except Exception as e:
            print(f"      ERROR calculating polynomial approximation for N={n_nodes}, Degree={degree}: {e}")
            poly_max_err = np.inf
            poly_mse = np.inf

        # Plot INDIVIDUAL Polynomial Approximation
        if poly_success:
            fig_poly, ax_poly = plt.subplots(figsize=(10, 6))
            plot_function(fine_x, func, nodes=nodes_uniform, y_lim=y_lim_plot,
                          function_name='Funkcja Oryginalna', nodes_name=f'{n_nodes} Węzłów',
                          title=f'Aproks. Wielomianowa (st. {degree}, {n_nodes} węzłów)',
                          alpha=0.7, linestyle=':')
            plot_function(fine_x, poly_approx_func, nodes=None, y_lim=y_lim_plot, color='red',
                          function_name=f'Aproks. Wielom. (st. {degree})', title='')
            plt.legend()
            plot_filename_poly = f'poly_approx_n{n_nodes}_deg{degree}.png'
            plt.savefig(os.path.join(PATH_TO_SAVE_UNIFORM_POLY_APPROX, plot_filename_poly))
            plt.close(fig_poly)
        elif degree < n_nodes: # Avoid logging skip message if skipped due to degree >= n_nodes check
             print(f"      Skipping individual polynomial plot for N={n_nodes}, Degree={degree} due to error.")

        # Record Polynomial Data
        data_records_approx.append({
            'Num Nodes': n_nodes, 'Degree': degree, 'Node Type': 'Uniform',
            'Approximation Type': 'Polynomial', 'Max Error': poly_max_err, 'MSE': poly_mse })

        # --- Trigonometric Approximation ---
        trig_success = False
        try:
            # Add check: degree must be <= n_nodes / 2 for trig approx? (Depends on implementation details, common constraint)
            # Adjust this check based on your get_trig_approximation_func implementation limits
            # Example check:
            # if degree > n_nodes // 2:
            #    print(f"      WARNING: Trig degree {degree} > n_nodes/2 ({n_nodes//2}). Skipping trig calculation.")
            #    trig_max_err = np.nan
            #    trig_mse = np.nan
            if degree >= 1: # Basic check
                 trig_approx_func = get_trig_approximation_func(nodes_uniform, degree)
                 trig_max_err = get_max_error(func, trig_approx_func, interval)
                 trig_mse = get_mean_squared_error(func, trig_approx_func, interval)
                 trig_success = True
                 trig_funcs_for_n[degree] = trig_approx_func
            else:
                 print(f"      Skipping trigonometric approximation for Degree={degree} (degree < 1).")
                 trig_max_err = np.nan
                 trig_mse = np.nan
        except Exception as e:
            print(f"      ERROR calculating trigonometric approximation for N={n_nodes}, Degree={degree}: {e}")
            trig_max_err = np.inf
            trig_mse = np.inf

        # Plot INDIVIDUAL Trigonometric Approximation
        if trig_success:
             fig_trig, ax_trig = plt.subplots(figsize=(10, 6))
             plot_function(fine_x, func, nodes=nodes_uniform, y_lim=y_lim_plot,
                           function_name='Funkcja Oryginalna', nodes_name=f'{n_nodes} Węzłów',
                           title=f'Aproks. Trygonometryczna (st. {degree}, {n_nodes} węzłów)',
                           alpha=0.7, linestyle=':')
             plot_function(fine_x, trig_approx_func, nodes=None, y_lim=y_lim_plot, color='blue',
                           function_name=f'Aproks. Trygon. (st. {degree})', title='')
             plt.legend()
             plot_filename_trig = f'trig_approx_n{n_nodes}_deg{degree}.png'
             plt.savefig(os.path.join(PATH_TO_SAVE_UNIFORM_TRIG_APPROX, plot_filename_trig))
             plt.close(fig_trig)
        # Avoid logging skip message again if skipped due to specific constraint checks
        elif degree >= 1 : # and degree <= n_nodes // 2 (if using that check):
             print(f"      Skipping individual trigonometric plot for N={n_nodes}, Degree={degree} due to error.")

        # Record Trigonometric Data
        data_records_approx.append({
            'Num Nodes': n_nodes, 'Degree': degree, 'Node Type': 'Uniform',
            'Approximation Type': 'Trigonometric', 'Max Error': trig_max_err, 'MSE': trig_mse })

    # --- Generate GROUPED Comparison Plots for the CURRENT n_nodes ---
    print(f"  Generating grouped comparison plots for N = {n_nodes}...")

    # ----- Polynomial Grouped Comparison -----
    if poly_funcs_for_n:
        available_poly_degrees = sorted(poly_funcs_for_n.keys())
        num_poly_plots = math.ceil(len(available_poly_degrees) / GROUP_SIZE)
        print(f"    Creating {num_poly_plots} polynomial comparison plot(s)...")

        for plot_idx in range(num_poly_plots):
            start_idx = plot_idx * GROUP_SIZE
            end_idx = start_idx + GROUP_SIZE
            degrees_in_group = available_poly_degrees[start_idx:end_idx]

            if not degrees_in_group: continue # Should not happen with ceil, but safe check

            plt.figure(figsize=(12, 7))
            group_str = f"St. {min(degrees_in_group)}-{max(degrees_in_group)}"
            plot_title_compare_poly = f'Porównanie Wielom. (N={n_nodes}, {group_str})'

            # Plot original function and nodes once per group plot
            plot_function(fine_x, func, nodes=nodes_uniform, y_lim=y_lim_plot,
                          function_name='Funkcja Oryginalna', nodes_name=f'{n_nodes} Węzłów',
                          title=plot_title_compare_poly, linestyle=':', color='black', alpha=0.7)

            # Plot polynomial approximations for this group with uniform colors
            for color_idx, deg in enumerate(degrees_in_group):
                p_func = poly_funcs_for_n[deg]
                color = COLORS[color_idx % len(COLORS)] # Cycle through defined colors
                plot_function(fine_x, p_func, nodes=None, y_lim=y_lim_plot, color=color,
                              function_name=f'Aproks. Wielom. (st. {deg})', title='')

            plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=GROUP_SIZE) # Legend below plot
            plt.grid(True)
            plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout
            comp_poly_filename = f'poly_comparison_n{n_nodes}_group{plot_idx+1}.png'
            plt.savefig(os.path.join(PATH_TO_SAVE_COMPARISON_POLY, comp_poly_filename))
            plt.close()
            print(f"      Saved grouped polynomial comparison plot: {comp_poly_filename}")
    else:
        print(f"    Skipping polynomial comparison plots for N={n_nodes} (no successful functions).")

    # ----- Trigonometric Grouped Comparison -----
    if trig_funcs_for_n:
        available_trig_degrees = sorted(trig_funcs_for_n.keys())
        num_trig_plots = math.ceil(len(available_trig_degrees) / GROUP_SIZE)
        print(f"    Creating {num_trig_plots} trigonometric comparison plot(s)...")

        for plot_idx in range(num_trig_plots):
            start_idx = plot_idx * GROUP_SIZE
            end_idx = start_idx + GROUP_SIZE
            degrees_in_group = available_trig_degrees[start_idx:end_idx]

            if not degrees_in_group: continue

            plt.figure(figsize=(12, 7))
            group_str = f"St. {min(degrees_in_group)}-{max(degrees_in_group)}"
            plot_title_compare_trig = f'Porównanie Trygon. (N={n_nodes}, {group_str})'

            # Plot original function and nodes once per group plot
            plot_function(fine_x, func, nodes=nodes_uniform, y_lim=y_lim_plot,
                          function_name='Funkcja Oryginalna', nodes_name=f'{n_nodes} Węzłów',
                          title=plot_title_compare_trig, linestyle=':', color='black', alpha=0.7)

            # Plot trigonometric approximations for this group with uniform colors
            for color_idx, deg in enumerate(degrees_in_group):
                t_func = trig_funcs_for_n[deg]
                color = COLORS[color_idx % len(COLORS)] # Cycle through defined colors
                plot_function(fine_x, t_func, nodes=None, y_lim=y_lim_plot, color=color,
                              function_name=f'Aproks. Trygon. (st. {deg})', title='')

            plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=GROUP_SIZE) # Legend below plot
            plt.grid(True)
            plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout
            comp_trig_filename = f'trig_comparison_n{n_nodes}_group{plot_idx+1}.png'
            plt.savefig(os.path.join(PATH_TO_SAVE_COMPARISON_TRIG, comp_trig_filename))
            plt.close()
            print(f"      Saved grouped trigonometric comparison plot: {comp_trig_filename}")
    else:
        print(f"    Skipping trigonometric comparison plots for N={n_nodes} (no successful functions).")


# --- Save Collected Data to CSV (after N_NODES loop) ---
df_approx = pd.DataFrame(data_records_approx)

# Format float columns before saving
float_cols_approx = ['Max Error', 'MSE']
if not df_approx.empty:
    df_approx[float_cols_approx] = df_approx[float_cols_approx].fillna(np.nan)
    df_approx[float_cols_approx] = df_approx[float_cols_approx].replace([np.inf, -np.inf], np.nan)
    if all(col in df_approx.columns for col in float_cols_approx):
        for col in float_cols_approx:
            df_approx[col] = df_approx[col].apply(lambda x: format_float_approx(x) if pd.notna(x) else 'nan')

csv_path_approx = os.path.join(PATH_TO_SAVE_DATA_APPROX, 'approximation_errors_combined.csv')
df_approx.to_csv(csv_path_approx, index=False)

print(f"\nBatch approximation process complete.")
print(f"Individual plots saved in '{PATH_TO_SAVE_UNIFORM_POLY_APPROX}' and '{PATH_TO_SAVE_UNIFORM_TRIG_APPROX}'.")
print(f"Grouped comparison plots saved in '{PATH_TO_SAVE_COMPARISON_POLY}' and '{PATH_TO_SAVE_COMPARISON_TRIG}'.")
print(f"CSV data saved to '{csv_path_approx}'.")

# Display the first few rows/last rows of the dataframe as a check
print("\n--- Sample of Collected Data ---")
print(df_approx.head())
print("...")
print(df_approx.tail())

In [ ]:
# --- Configuration for Node Comparison Plot ---
# Choose the specific DEGREEs (m) to keep constant
M_TO_PLOT_LIST = [5, 10, 15, 20]

# Choose the specific list of NODE counts (n) for EACH degree in M_TO_PLOT_LIST
# Ensure the number of inner lists matches the length of M_TO_PLOT_LIST
N_NODES_LISTS = [
  [11, 15, 20, 25, 30, 50],           # For M=5
  [21, 30, 50, 70, 90, 110],           # For M=10
  [31, 40, 50, 70, 90, 110],          # For M=15
  [41, 50, 60, 70, 90, 110],              # For M=20 (Adjusted example length)                # For M=50 (Adjusted example length)
]

# Grouping and Coloring for Node Comparison Plots
NODE_GROUP_SIZE = 3 # How many 'n' values per plot
NODE_COLORS = ['green', 'red', 'blue', '#d62728', '#9467bd', '#8c564b'] # Example color cycle

# --- Check Configuration Lengths ---
if len(M_TO_PLOT_LIST) != len(N_NODES_LISTS):
    raise ValueError("Error: The number of degrees in M_TO_PLOT_LIST must match the number of node lists in N_NODES_LISTS.")

# --- Outer Loop for Each Degree (M) ---
print("Starting batch node comparison plot generation...")

# Common plot elements
fine_x_compare_n = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES)
y_lim_compare_n = (-10, 40) # Consistent y-limits

for m_idx, current_m in enumerate(M_TO_PLOT_LIST):
    current_n_nodes_list = N_NODES_LISTS[m_idx]

    print(f"\n===== Processing for Fixed Degree M = {current_m} =====")
    print(f"  Using N values: {current_n_nodes_list}")

    # Define specific paths for these new comparison plots FOR THE CURRENT M
    PATH_TO_SAVE_NODE_COMPARISON_POLY = os.path.join(PATH_TO_SAVE_MAIN_APPROX, f'poly_node_comparison_m{current_m}_grouped')
    PATH_TO_SAVE_NODE_COMPARISON_TRIG = os.path.join(PATH_TO_SAVE_MAIN_APPROX, f'trig_node_comparison_m{current_m}_grouped')

    # --- Create Directories for the current M ---
    os.makedirs(PATH_TO_SAVE_NODE_COMPARISON_POLY, exist_ok=True)
    os.makedirs(PATH_TO_SAVE_NODE_COMPARISON_TRIG, exist_ok=True)

    # --- Pre-calculate Approximations for the current M ---
    print(f"  Pre-calculating approximations for M = {current_m}...")

    # Reset dictionaries for each M value
    poly_funcs_by_n = {}
    trig_funcs_by_n = {}

    for n_nodes in current_n_nodes_list:
        print(f"    Processing N = {n_nodes} nodes...")
        # Generate nodes
        try:
            nodes_uniform = generate_function_uniform_nodes(n_nodes, interval, func)
        except Exception as e:
            print(f"      ERROR generating nodes for N={n_nodes}: {e}. Skipping N.")
            continue

        # Calculate Polynomial Approximation for current_m
        if current_m >= n_nodes:
             print(f"      WARNING: Polynomial degree {current_m} >= number of nodes {n_nodes}. Skipping poly.")
        else:
            try:
                poly_approx_func = get_poly_approximation_func(nodes_uniform, current_m)
                poly_funcs_by_n[n_nodes] = poly_approx_func
                # print(f"      Successfully calculated polynomial approx for N={n_nodes}") # Verbose
            except Exception as e:
                print(f"      ERROR calculating polynomial approx for N={n_nodes}, M={current_m}: {e}")

        # Calculate Trigonometric Approximation for current_m
        if current_m < 1:
            print(f"      WARNING: Trig degree {current_m} < 1. Skipping trig.")
        # Optional: Add check like current_m > n_nodes // 2 if relevant
        # elif current_m > n_nodes // 2:
        #    print(f"      WARNING: Trig degree {current_m} > n_nodes/2 ({n_nodes//2}). Skipping trig.")
        else:
             try:
                 trig_approx_func = get_trig_approximation_func(nodes_uniform, current_m)
                 trig_funcs_by_n[n_nodes] = trig_approx_func
                 # print(f"      Successfully calculated trigonometric approx for N={n_nodes}") # Verbose
             except Exception as e:
                 print(f"      ERROR calculating trigonometric approx for N={n_nodes}, M={current_m}: {e}")


    # --- Generate GROUPED Node Comparison Plots for the current M ---
    print(f"\n  Generating grouped node comparison plots for M = {current_m}...")

    # ----- Polynomial Node Comparison (Grouped) -----
    if poly_funcs_by_n:
        available_poly_n_nodes = sorted(poly_funcs_by_n.keys())
        num_poly_plots = math.ceil(len(available_poly_n_nodes) / NODE_GROUP_SIZE)
        print(f"    Creating {num_poly_plots} polynomial node comparison plot(s)...")

        for plot_idx in range(num_poly_plots):
            start_idx = plot_idx * NODE_GROUP_SIZE
            end_idx = start_idx + NODE_GROUP_SIZE
            n_nodes_in_group = available_poly_n_nodes[start_idx:end_idx]

            if not n_nodes_in_group: continue

            plt.figure(figsize=(12, 7))
            group_n_str = f"N={min(n_nodes_in_group)}-{max(n_nodes_in_group)}"
            plot_title_compare_poly = f'Porównanie Wielom. dla st. {current_m} ({group_n_str})'

            plot_function(fine_x_compare_n, func, nodes=None, y_lim=y_lim_compare_n,
                          function_name='Funkcja Oryginalna',
                          title=plot_title_compare_poly, linestyle=':', color='black', alpha=0.7)

            for color_idx, n_nodes_val in enumerate(n_nodes_in_group):
                p_func = poly_funcs_by_n[n_nodes_val]
                color = NODE_COLORS[color_idx % len(NODE_COLORS)]
                plot_function(fine_x_compare_n, p_func, nodes=None, y_lim=y_lim_compare_n, color=color,
                              function_name=f'N = {n_nodes_val}', title='')

            plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=NODE_GROUP_SIZE)
            plt.grid(True)
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            comp_poly_filename = f'poly_node_compare_m{current_m}_group{plot_idx+1}.png'
            plt.savefig(os.path.join(PATH_TO_SAVE_NODE_COMPARISON_POLY, comp_poly_filename))
            plt.close()
            print(f"      Saved polynomial node comparison plot: {comp_poly_filename}")
    else:
        print(f"    Skipping polynomial node comparison plots for M={current_m} (no successful functions).")

    # ----- Trigonometric Node Comparison (Grouped) -----
    if trig_funcs_by_n:
        available_trig_n_nodes = sorted(trig_funcs_by_n.keys())
        num_trig_plots = math.ceil(len(available_trig_n_nodes) / NODE_GROUP_SIZE)
        print(f"    Creating {num_trig_plots} trigonometric node comparison plot(s)...")

        for plot_idx in range(num_trig_plots):
            start_idx = plot_idx * NODE_GROUP_SIZE
            end_idx = start_idx + NODE_GROUP_SIZE
            n_nodes_in_group = available_trig_n_nodes[start_idx:end_idx]

            if not n_nodes_in_group: continue

            plt.figure(figsize=(12, 7))
            group_n_str = f"N={min(n_nodes_in_group)}-{max(n_nodes_in_group)}"
            plot_title_compare_trig = f'Porównanie Trygon. dla st. {current_m} ({group_n_str})'

            plot_function(fine_x_compare_n, func, nodes=None, y_lim=y_lim_compare_n,
                          function_name='Funkcja Oryginalna',
                          title=plot_title_compare_trig, linestyle=':', color='black', alpha=0.7)

            for color_idx, n_nodes_val in enumerate(n_nodes_in_group):
                t_func = trig_funcs_by_n[n_nodes_val]
                color = NODE_COLORS[color_idx % len(NODE_COLORS)]
                plot_function(fine_x_compare_n, t_func, nodes=None, y_lim=y_lim_compare_n, color=color,
                              function_name=f'N = {n_nodes_val}', title='')

            plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=NODE_GROUP_SIZE)
            plt.grid(True)
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            comp_trig_filename = f'trig_node_compare_m{current_m}_group{plot_idx+1}.png'
            plt.savefig(os.path.join(PATH_TO_SAVE_NODE_COMPARISON_TRIG, comp_trig_filename))
            plt.close()
            print(f"      Saved trigonometric node comparison plot: {comp_trig_filename}")
    else:
        print(f"    Skipping trigonometric node comparison plots for M={current_m} (no successful functions).")

# --- End of Outer Loop ---
print("\n==============================================")
print("All node comparison plot generation complete.")
print(f"Plots saved in subdirectories within: {PATH_TO_SAVE_MAIN_APPROX}")

In [ ]:

# --- Configuration ---
N_NODES = [i for i in range(3, 52, 2)] # Adjusted range for a potentially more informative heatmap
# M_DEGREES_LOOP_STEP = 3 # Keep track of the step used for degrees

PATH_TO_SAVE_HEATMAPS = os.path.join('.', 'heatmaps_approximation') # Renamed for clarity

# --- Cleanup and Directory Creation ---
if os.path.exists(PATH_TO_SAVE_HEATMAPS):
  shutil.rmtree(PATH_TO_SAVE_HEATMAPS)
os.makedirs(PATH_TO_SAVE_HEATMAPS, exist_ok=True)

# --- Data Collection Setup ---
poly_data_list = []
trig_data_list = []

# --- Main Loop for Data Generation ---
print("Starting batch approximation process for heatmaps...")

# Make sure interval, func, NUMBER_OF_PROBES are defined from previous cells
# interval = (-5.0, 5.0)
# func = lambda x: x**2 - m * np.cos((np.pi * x) / k) # Assuming m, k are defined
# NUMBER_OF_PROBES = 1000

for n_nodes in N_NODES:
    print(f"  Processing N = {n_nodes} nodes...")
    # Check if n_nodes is large enough for any degrees in the loop
    nodes_uniform = generate_function_uniform_nodes(n_nodes, interval, func)

    # Determine degrees based on n_nodes for this specific task
    # Note: Degrees will depend on n_nodes. The resulting grid might not be perfectly rectangular.
    degrees_to_test = range(1, int(n_nodes/2)+1, 1) # Iterate up to n_nodes-1 with step 3

    if not list(degrees_to_test):
         print(f"No degrees to test in range(2, {n_nodes}, 3) for N={n_nodes}.")
         continue

    for degree in degrees_to_test:

        # --- Polynomial ---
        # poly_approx_func = get_poly_approximation_func(nodes_uniform, degree)
        # poly_max_err = get_max_error(func, poly_approx_func, interval)
        # poly_mse = get_mean_squared_error(func, poly_approx_func, interval)
        # poly_data_list.append({
        #     'n': n_nodes,
        #     'm': degree,
        #     'Max Error': poly_max_err,
        #     'MSE': poly_mse
        # })
        # --- Trigonometric ---
        # Ensure degree is valid for trig (m >= 1 usually, our loop starts at 2)
        trig_approx_func = get_trig_approximation_func(nodes_uniform, degree)
        trig_max_err = get_max_error(func, trig_approx_func, interval)
        trig_mse = get_mean_squared_error(func, trig_approx_func, interval)
        trig_data_list.append({
            'n': n_nodes,
            'm': degree,
            'Max Error': trig_max_err,
            'MSE': trig_mse
        })

# --- Create DataFrames ---
data_records_poly = pd.DataFrame(poly_data_list)
data_records_trig = pd.DataFrame(trig_data_list)

# Replace inf with NaN for better heatmap handling if desired, although heatmap handles inf too
data_records_poly.replace([np.inf, -np.inf], np.nan, inplace=True)
data_records_trig.replace([np.inf, -np.inf], np.nan, inplace=True)

# --- Create and Save Heatmaps ---
print("\nGenerating heatmaps...")

# Optional: Use LogNorm if errors span many orders of magnitude
# use_log_scale = True
use_log_scale = True # Set to True if needed

def create_and_save_heatmap(df, value_col, title, filename):
    if df.empty or value_col not in df.columns or df[value_col].isnull().all():
        print(f"Skipping heatmap '{title}': DataFrame is empty, missing column, or all NaNs.")
        return
    try:
        pivot_df = df.pivot(index='n', columns='m', values=value_col)

        plt.figure(figsize=(20, 16))
        norm = None
        fmt = ".2f" # Default format

        # Apply LogNorm if errors are positive and log scale is requested
        if use_log_scale:
             # Filter out non-positive values for log scale compatibility
            valid_data = pivot_df[pivot_df > 0]
            if not valid_data.empty:
                 vmin = valid_data.min().min()
                 vmax = valid_data.max().max()
                 if vmin > 0 and vmax > vmin: # Ensure valid range for LogNorm
                     norm = mcolors.LogNorm(vmin=vmin, vmax=vmax)
                     fmt = ".1e" # Scientific notation is often better with log scale
                 else:
                    print(f"  Cannot use log scale for {title}: Invalid data range after filtering non-positive values.")
            else:
                 print(f"  Cannot use log scale for {title}: No positive data found.")


        sns.heatmap(pivot_df, annot=True, annot_kws={"size": 7}, fmt=fmt, cmap="viridis_r", norm=norm, cbar=True)
        # colorbar = ax.collections[0].colorbar

        # # Set specific ticks and labels
        # values_to_mark = [0.01, 0.1, 1, 10, 100]
        # colorbar.set_ticks(values_to_mark)
        
        # def scientific_notation(x, pos):
        #   exponent = int(np.log10(x))
        #   return r'$10^{%d}$' % exponent

        # # Apply the formatter
        # colorbar.ax.yaxis.set_major_formatter(FuncFormatter(scientific_notation))

        # colorbar.set_ticklabels([f'{v:.1f}' for v in values_to_mark])

        plt.title(title)
        plt.xlabel("Stopień aproksymacji (m)")
        plt.ylabel("Liczba węzłów (n)")
        plt.tight_layout()
        plt.savefig(os.path.join(PATH_TO_SAVE_HEATMAPS, filename))
        plt.close() # Close the figure to free memory
        print(f"  Saved heatmap: {filename}")
    except Exception as e:
        print(f"  Failed to generate heatmap '{title}': {e}")
        plt.close() # Ensure figure is closed even if error occurs

# Generate the four heatmaps
create_and_save_heatmap(data_records_poly, 'Max Error', 'Błąd Maksymalny - Aproksymacja Wielomianowa', 'heatmap_poly_max_error.png')
create_and_save_heatmap(data_records_poly, 'MSE', 'Błąd Średniokwadratowy (MSE) - Aproks. Wielomianowa', 'heatmap_poly_mse.png')
create_and_save_heatmap(data_records_trig, 'Max Error', 'Błąd Maksymalny - Aproksymacja Trygonometryczna', 'heatmap_trig_max_error.png')
create_and_save_heatmap(data_records_trig, 'MSE', 'Błąd Średniokwadratowy (MSE) - Aproks. Trygonometryczna', 'heatmap_trig_mse.png')

print("\nHeatmap generation complete.")
print(f"Heatmaps saved in: {PATH_TO_SAVE_HEATMAPS}")

# Display sample data (optional)
print("\nSample Polynomial Data:")
print(data_records_poly.head())
print("\nSample Trigonometric Data:")
print(data_records_trig.head())

poly_csv_path = os.path.join(PATH_TO_SAVE_HEATMAPS, 'data_poly_results.csv')
trig_csv_path = os.path.join(PATH_TO_SAVE_HEATMAPS, 'data_trig_results.csv')

data_records_poly.to_csv(poly_csv_path, index=False)
data_records_trig.to_csv(trig_csv_path, index=False)

print(f"\nPolynomial data saved to: {poly_csv_path}")
print(f"Trigonometric data saved to: {trig_csv_path}")




In [ ]:
# --- Configuration for Comparison Plot ---
N_TO_PLOT = 51  # Choose the specific number of nodes (must be one of the values used before or a new one)
DEGREES_TO_PLOT = [20] # Choose the specific degrees to compare on the plot
# COLORS = ['green', 'red', 'blue']
COLORS = ['blue']

# Optional: Define a specific path to save this comparison plot
PATH_TO_SAVE_COMPARISON_PLOTS = os.path.join('.', 'img_comparison')
os.makedirs(PATH_TO_SAVE_COMPARISON_PLOTS, exist_ok=True)

# --- Generate Nodes (only once) ---
print(f"Generating comparison plot for N = {N_TO_PLOT} nodes.")
try:
    nodes_uniform_compare = generate_function_uniform_nodes(N_TO_PLOT, interval, func)
    nodes_generated = True
except Exception as e:
    print(f"  ERROR generating nodes for N={N_TO_PLOT}: {e}")
    nodes_generated = False

# --- Plotting Setup ---
if nodes_generated:
    fine_x_compare = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES)
    y_lim_compare = (-10, 40) # Use consistent y-limits if desired

    plt.figure(figsize=(12, 7)) # Create a new figure for the comparison plot

    # Plot the Original Function and Nodes FIRST
    plot_title_compare = f'Porównanie Aproksymacji Wielomianowych (N={N_TO_PLOT})'
    plot_function(fine_x_compare, func, nodes=nodes_uniform_compare, y_lim=y_lim_compare,
                  function_name='Funkcja Oryginalna', nodes_name=f'{N_TO_PLOT} Węzłów (Równoodl.)',
                  title=plot_title_compare,
                  linestyle=':', color='black', alpha=0.8) # Make original function distinct

    # --- Loop Through Degrees and Plot Approximations ---
    print(f"  Calculating and plotting approximations for degrees: {DEGREES_TO_PLOT}")
    for i,degree in enumerate(DEGREES_TO_PLOT):
        print(f"    Processing Degree = {degree}...")
        try:
            # Calculate the approximation for the current degree using the SAME nodes
            poly_approx_func_compare = get_poly_approximation_func(nodes_uniform_compare, degree)

            # Plot the approximation function ON THE SAME FIGURE
            # Use plot_function but without nodes and an empty title to reuse axes
            plot_function(fine_x_compare, poly_approx_func_compare, nodes=None, # No nodes here
                          y_lim=y_lim_compare, color=COLORS[i],
                          function_name=f'Aproks. Wielom. (st. {degree})',
                          title='') # Empty title to avoid overwriting

            print(f"      Successfully plotted Degree = {degree}")

        except Exception as e:
            print(f"      ERROR calculating/plotting polynomial approximation for Degree={degree}: {e}")
            # Optionally, you could add a note to the plot or skip this degree

    # --- Finalize Plot ---
    plt.legend() # Add legend to identify all lines
    plt.grid(True)

    # Save the comparison plot
    comparison_plot_filename = f'poly_comparison_n{N_TO_PLOT}_degrees_{"_".join(map(str, DEGREES_TO_PLOT))}.png'
    plt.savefig(os.path.join(PATH_TO_SAVE_COMPARISON_PLOTS, comparison_plot_filename))
    print(f"\nComparison plot saved as: {os.path.join(PATH_TO_SAVE_COMPARISON_PLOTS, comparison_plot_filename)}")

    plt.show() # Display the plot

else:
    print("Skipping comparison plot due to node generation error.")

In [ ]:
# TRIG COMPARISON PLOT
N_TO_PLOT = 21  # Choose the specific number of nodes (must be one of the values used before or a new one)
DEGREES_TO_PLOT = [8,9,10] # Choose the specific degrees to compare on the plot
COLORS = ['green', 'red', 'blue']
# COLORS = ['blue']
# COLORS = ['red']

# Optional: Define a specific path to save this comparison plot
PATH_TO_SAVE_COMPARISON_PLOTS = os.path.join('.', 'img_comparison')
os.makedirs(PATH_TO_SAVE_COMPARISON_PLOTS, exist_ok=True)

# --- Generate Nodes (only once) ---
print(f"Generating comparison plot for N = {N_TO_PLOT} nodes.")
try:
    nodes_uniform_compare = generate_function_uniform_nodes(N_TO_PLOT, interval, func)
    nodes_generated = True
except Exception as e:
    print(f"  ERROR generating nodes for N={N_TO_PLOT}: {e}")
    nodes_generated = False

# --- Plotting Setup ---
if nodes_generated:
    fine_x_compare = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES)
    y_lim_compare = (-10, 40) # Use consistent y-limits if desired

    plt.figure(figsize=(12, 7)) # Create a new figure for the comparison plot

    # Plot the Original Function and Nodes FIRST
    plot_title_compare = f'Porównanie Aproksymacji Wielomianowych (N={N_TO_PLOT})'
    plot_function(fine_x_compare, func, nodes=nodes_uniform_compare, y_lim=y_lim_compare,
                  function_name='Funkcja Oryginalna', nodes_name=f'{N_TO_PLOT} Węzłów (Równoodl.)',
                  title=plot_title_compare,
                  linestyle=':', color='black', alpha=0.8) # Make original function distinct

    # --- Loop Through Degrees and Plot Approximations ---
    print(f"  Calculating and plotting approximations for degrees: {DEGREES_TO_PLOT}")
    for i,degree in enumerate(DEGREES_TO_PLOT):
        print(f"    Processing Degree = {degree}...")
        try:
            # Calculate the approximation for the current degree using the SAME nodes
            poly_approx_func_compare = get_trig_approximation_func(nodes_uniform_compare, degree)

            # Plot the approximation function ON THE SAME FIGURE
            # Use plot_function but without nodes and an empty title to reuse axes
            plot_function(fine_x_compare, poly_approx_func_compare, nodes=None, # No nodes here
                          y_lim=y_lim_compare, color=COLORS[i],
                          function_name=f'Aproks. Wielom. (st. {degree})',
                          title='') # Empty title to avoid overwriting

            print(f"      Successfully plotted Degree = {degree}")

        except Exception as e:
            print(f"      ERROR calculating/plotting polynomial approximation for Degree={degree}: {e}")
            # Optionally, you could add a note to the plot or skip this degree

    # --- Finalize Plot ---
    plt.legend() # Add legend to identify all lines
    plt.grid(True)

    # Save the comparison plot
    comparison_plot_filename = f'poly_comparison_n{N_TO_PLOT}_degrees_{"_".join(map(str, DEGREES_TO_PLOT))}.png'
    plt.savefig(os.path.join(PATH_TO_SAVE_COMPARISON_PLOTS, comparison_plot_filename))
    print(f"\nComparison plot saved as: {os.path.join(PATH_TO_SAVE_COMPARISON_PLOTS, comparison_plot_filename)}")

    plt.show() # Display the plot

else:
    print("Skipping comparison plot due to node generation error.")